# Transfer Learning


### Celem jest zapoznanie się z dostępnymi modelami (architekturami).

### Modele
Jak widać Keras ma trochę dostępnych modeli. Jeśli chcesz poczytać więcej o każdym z nich, poniżej są linki:

- [ResNet50: Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
- [VGG16: Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)
- [VGG19: Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)
- [InceptionResNetV2: Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning](https://arxiv.org/pdf/1602.07261.pdf)
- [InceptionV3: Rethinking the Inception Architecture for Computer Vision](http://arxiv.org/abs/1512.00567)
- [MobileNet: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/pdf/1704.04861.pdf)
- [Xception: Deep Learning with Depthwise Separable Convolutions](https://arxiv.org/abs/1610.02357)

Weźmiemy na początek jeden z prostszych, ale dość popularny - **VGG16**.

*Swoją drogą*, wczytując model, mamy flagę `include_top=False`, która oznacza, że obcinamy warstwę klasyfikatora (czyli `fully-connected layer`). Na początek wczytamy całość (razem z `fully-connected layers`).

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D

from keras.optimizers import Adam, SGD

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201

from keras.preprocessing import image

import numpy as np
np.random.seed(0)

In [ ]:
import keras.applications
dir(keras.applications)

## VGG16

In [ ]:
model = VGG16(weights='imagenet', include_top=True)
model.summary()

Jaki widać VGG16 ma ponad 138M parametrów (bez `fully-connected` ma 14M i trochę, czyli 10 razy mniej). Domyślnie wszystkie warstwy są "odmrożone" (zwróć uwagę na ostatnią linię poprzedniej komórki: `Non-trainable params: 0`).

Do warstw można dostać się w taki sposób: `model.layers`. Każda warstwa ma atrybut `trainable`, który zwraca `True` lub `False`. Sprawdźmy to.

In [ ]:
for it, layer in enumerate(model.layers):
    print(it, layer.trainable)

Pierwsza warstwa już jest ustawiona na `False`, bo jest to `InputLayer`. Załóżmy, że chcemy trenować (ang. *fine tuning*) tylko ostatni (5. blok), który składa się z czterech warstw (niskopoziomowych):

```
block5_conv1 (Conv2D)     (None, None, None, 512)   2359808   
block5_conv2 (Conv2D)     (None, None, None, 512)   2359808   
block5_conv3 (Conv2D)     (None, None, None, 512)   2359808   
block5_pool (MaxPool2D)   (None, None, None, 512)   0 
```


W takiej sytuacji można dla reszty warstw ustawić parametr `trainable` na `False`.

In [ ]:
for it, layer in enumerate(model.layers[:-8]):
    layer.trainable = False
for it, layer in enumerate(model.layers[-4:]):
    layer.trainable = False

Sprawdzamy całość i potem jeszcze `summary` (zwracamy uwagę na ostatnią linię - `Non-trainable params`).

In [ ]:
for it, layer in enumerate(model.layers):
    print(it, layer.name, layer.trainable)

In [ ]:
model.summary()

In [ ]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()

## Przypadki zadań 
W dużym uproszczeniu wszystkie zadania w Deep Learning może klasyfikować wg dwóch wymiarów:
![](../images/four_cases.png)

A teraz przejdziemy po kolei po każdym przypadku.

### (1) Danych jest sporo i zadanie bardzo się różni.

W takiej sytuacji tak naprawdę uczymy naszą sieć od zera (czyli `ImageNet` trafia do tego przypadku, kiedy mamy miliony zdjęć i zupełnie nowe zadanie).

### (2) Danych jest dużo, ale zadanie jest podobne.

Skoro mamy dużo danych, to możemy uczyć się nawet od zera (przynajmniej nie powinien nam grozić `overfitting`). Z drugiej strony, skoro zadanie jest podobne, to warto wykorzystać już wyliczone wagi. 

Dlatego strategia jest taka - odmrażamy wszystkie warstwy (`trainable=True`), to jest opcja domyślna. Jedynie jeszcze można przerobić `fully-connected layer` - chyba, że to też nam odpowiada (np. `ImageNet` ma 1000 klas na wyjściu).

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers[1:]: #input layer ma być na False
    layer.trainable = True


model = Sequential([
    base_model,
    
    Flatten(), #<= bridge between conv layers and full connected layers
    
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
    
])

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

Sprawdzamy, jak to jest.

In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable)
    
model.summary()

Zwróć uwagę, że VGG16 teraz jest pokazany jako warstwa - jest to dość wygodne, w `summary` widać tylko to, co jest ważne (nasze).

### (3) Danych jest mało, ale i zadanie jest podobne.

W sytuacji gdy danych jest mało lub wcale ich nie ma, to musimy użyć takiego modelu, jaki jest (właśnie to już zrobiliśmy w punkcie `2`) albo zamrozić wszystkie warstwy konwolucyjne i wyrzucić lub odmrozić `fully-connected layers`.

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers[1:]: #input layer ma być na False
    layer.trainable = False #!!!! zwróć uwagę, zamrażamy wszystkie warstwy


model = Sequential([
    base_model,
    
    Flatten(), #<= bridge between conv layers and full connected layers
    
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
    
])

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

Sprawdzamy, jak to jest:

In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable)
    
model.summary()

W drugim przypadku było `Trainable params: 15,250,250`, a w tym przypadku jest: `Trainable params: 535,562`. Trenujemy tylko `fully-connected layer` i nawet mając mało danych jest szansa, że nie będziemy "overfitować".

### (4) Danych jest mało i zadanie (bardzo) się różni.
To jest najgorszy przypadek i niestety dość częsty :) Trzeba sobie jakoś z tym radzić. W tej sytuacji odcięcie tylko `full-connected layer` to raczej za mało - trzeba odmrozić coś więcej (kilka lub więcej warstw konwolucyjnych). Natomiast możemy tutaj zastosować dość sprytne zagranie, które zwykle sprawdza się w praktyce.

Na początek zamrażamy wszystkie warstwy konwolucyjne i trenujemy tylko `fully-connected layer`, a następnie odmrażamy wszystko, tylko trenujemy z mniejszym krokiem.

### Faza pierwsza
Zamrażamy wszystkie warstwy konwolucyjne i trenujemy tylko `fully-connected layer`.

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

for layer in base_model.layers[1:]: #input layer ma być na False
    layer.trainable = False #!!!! zwróć uwagę, zamrażamy wszystkie warstwy


model = Sequential([
    base_model,
    
    Flatten(), #<= bridge between conv layers and full connected layers
    
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
    
])

optimizer = Adam(0.001, decay=0.0003) #wartości są przykładowe, zwróć uwagę tylko na rząd
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

for layer in model.layers:
    print(layer.name, layer.trainable)
    
model.summary()

### Faza druga

Odmrażamy wszystkie lub część warstw z VGG. Swoją drogą, żeby się tam dostać trzeba odwołać się do pierwszej warstwy `model.layers[0]`.

In [ ]:
for layer in model.layers[0].layers[10:]: 
    layer.trainable = True

optimizer = Adam(0.0001, decay=0.00000001) #wartości są przykładowe, zwróć uwagę tylko na rząd
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
for layer in model.layers[0].layers:
    print(layer.name, layer.trainable)
    
model.summary()

## Zadanie 7.3.1
1. Do której grupy (1,2,3 lub 4) przypiszesz zadanie 6.5 (znaki drogowe)?
2. Wyjaśnij, dlaczego tak myślisz. 


## Odpowiedzi 7.3.1
1. 
2. 

## Zadanie 7.3.2 (zaawansowane)
Spróbuj użyć `VGG` lub `ResNet` dla predykcji znaków drogowych.

In [ ]:
## YOUR CODE HERE

## Przydatne linki:
1. [CS231n: Transfer Learning](http://cs231n.github.io/transfer-learning/)
2. [Learning Keras by Implementing the VGG Network From Scratch](https://hackernoon.com/learning-keras-by-implementing-vgg16-from-scratch-d036733f2d5)
3. [Experiments on different loss configurations for style transfer](https://towardsdatascience.com/experiments-on-different-loss-configurations-for-style-transfer-7e3147eda55e)
4. [Transfer Learning using Keras](https://medium.com/@14prakash/transfer-learning-using-keras-d804b2e04ef8)
5. [A Comprehensive Hands-on Guide to Transfer Learning with Real-World Applications in Deep Learning)](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a)